In [21]:
import cv2
import numpy as np
import requests

In [2]:
face_cascade=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
def detect(gray,frame):
    
    faces=face_cascade.detectMultiScale(gray,1.3,5)
    for(x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        cv2.putText(frame,'face',(x,y),cv2.FONT_HERSHEY_COMPLEX,2,(255,0,0))
    return frame

In [ ]:
r = requests.get('http://192.168.1.104:8081/', stream=True)
if(r.status_code == 200):
    bytes = bytes()
    for chunk in r.iter_content(chunk_size=1024):
        bytes += chunk
        a = bytes.find(b'\xff\xd8')
        b = bytes.find(b'\xff\xd9')
        if a != -1 and b != -1:
            jpg = bytes[a:b+2]
            bytes = bytes[b+2:]
            i = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
            gray=cv2.cvtColor(i,cv2.COLOR_BGR2GRAY)#convert color to gray
            cv2.imshow('i', detect(gray,i))
            if cv2.waitKey(1) == 27:
                exit(0)

/anaconda3/envs/tensorflow/lib/python3.6/site-packages/ipykernel_launcher.py:11: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
######################LANDMARK DETECTION####################################

In [11]:
import argparse
import dlib

In [12]:
ap=argparse.ArgumentParser()



In [13]:
ap.add_argument("-p", "--shape-predictor", required=True,
	help="path to facial landmark predictor")

_StoreAction(option_strings=['-p', '--shape-predictor'], dest='shape_predictor', nargs=None, const=None, default=None, type=None, choices=None, help='path to facial landmark predictor', metavar=None)

In [14]:
ap.add_argument("-i", "--image", required=True,
	help="path to input image")

_StoreAction(option_strings=['-i', '--image'], dest='image', nargs=None, const=None, default=None, type=None, choices=None, help='path to input image', metavar=None)

In [18]:
args={'shape_predictor':'shape_predictor_68_face_landmarks.dat','image':'download-1.jpg'}

In [19]:
detector=dlib.get_frontal_face_detector()

In [20]:
predictor=dlib.shape_predictor(args['shape_predictor'])

In [29]:
image=cv2.imread(args['image'])

In [23]:
image.shape

(196, 257, 3)

In [31]:
image=cv2.resize(image,(500,500))

In [32]:
image.shape

(500, 500, 3)

In [27]:
import matplotlib.pyplot as plt

In [40]:
def shape_to_np(shape, dtype="int"):
	# initialize the list of (x, y)-coordinates
	coords = np.zeros((68, 2), dtype=dtype)
 
	# loop over the 68 facial landmarks and convert them
	# to a 2-tuple of (x, y)-coordinates
	for i in range(0, 68):
		coords[i] = (shape.part(i).x, shape.part(i).y)
 
	# return the list of (x, y)-coordinates
	return coords

In [41]:
def rect_to_bb(rect):
	# take a bounding predicted by dlib and convert it
	# to the format (x, y, w, h) as we would normally do
	# with OpenCV
	x = rect.left()
	y = rect.top()
	w = rect.right() - x
	h = rect.bottom() - y
 
	# return a tuple of (x, y, w, h)
	return (x, y, w, h)

In [35]:
image=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)

In [37]:
rects=detector(image,1)

In [ ]:
for (i,rect) in enumerate(rects):
    shape=predictor(image,rect)
    shape=shape_to_np(shape)
    
    (x,y,w,h)=rect_to_bb(rect)
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    
    for (x, y) in shape:
        cv2.circle(image, (x, y), 1, (0, 0, 255), -1)

cv2.imshow("Output", image)
cv2.waitKey(0)
